# 对象属性辅助函数

```{topic} 核心
使用 TVM 中的 `AttrsNode` 类和相关宏来声明和使用具有默认值和边界检查的命名属性。
```

示例：

```c++
struct MyAttrs : public tvm::AttrsNode<MyAttrs> {
    float learning_rate;
    int num_hidden;
    String name;
    // 声明属性字段和头文件
    TVM_DECLARE_ATTRS(MyAttrs, "attrs.MyAttrs") {
    TVM_ATTR_FIELD(num_hidden).set_lower_bound(1);
    TVM_ATTR_FIELD(learning_rate).set_default(0.01f);
    TVM_ATTR_FIELD(name).set_default("hello");
    }
};
// 在 cc 文件中注册
TVM_REGISTER_NODE_TYPE(MyAttrs);
```

首先，定义了名为 `MyAttrs` 的结构体，它继承自 `tvm::AttrsNode<MyAttrs>`。这个结构体包含了三个属性：`learning_rate`、`num_hidden` 和 `name`。在头文件中，可以使用 `TVM_DECLARE_ATTRS` 宏来声明这些属性。在这个例子中，声明了三个属性字段：`num_hidden`、`learning_rate` 和 `name`。每个字段都使用了 `TVM_ATTR_FIELD` 宏来指定属性的名称和一些选项。

- 对于 `num_hidden` 字段，使用 `set_lower_bound(1)` 来设置它的下界为 `1`，表示该属性的有效取值范围是大于等于 $1$ 的整数。
- 对于 `learning_rate` 字段，使用 `set_default(0.01f)` 来设置它的默认值为 `0.01`。这意味着如果在代码中没有显式地给 `learning_rate` 赋值，那么它将被初始化为 `0.01`。
- 对于 `name` 字段，使用 `set_default("hello")` 来设置它的默认值为 `"hello"`。同样地，如果在代码中没有显式地给 `name` 赋值，那么它将被初始化为 `"hello"`。

在源文件（通常是 C++）中，需要注册这个新的节点类型。使用 `TVM_REGISTER_NODE_TYPE(MyAttrs)` 宏可以将 `MyAttrs` 节点类型注册到 TVM 中，使其可以在后续的编译和执行过程中被识别和使用。

总结起来，这段代码演示了如何使用TVM的 `AttrsNode` 类和相关宏来声明和使用具有默认值和边界检查的命名属性。通过这种方式，可以更方便地管理和使用模型的属性信息。

## `TVM_DECLARE_ATTRS` 和 `TVM_ATTR_FIELD`

```c++
#define TVM_DECLARE_ATTRS(ClassName, TypeKey)                    \
  static constexpr const char* _type_key = TypeKey;              \
  TVM_DECLARE_FINAL_OBJECT_INFO(ClassName, ::tvm::BaseAttrsNode) \
  template <typename FVisit>                                     \
  void _tvm_VisitAttrs(FVisit& _tvm_fvisit)  // NOLINT(*)
```

```c++
#define TVM_ATTR_FIELD(FieldName) _tvm_fvisit(#FieldName, &FieldName)
```

这段代码定义了两个宏，用于在 TVM 中声明属性函数和属性字段。

第一个宏 `TVM_DECLARE_ATTRS` 用于声明属性函数。它接受两个参数：`ClassName` 表示类的名称，`TypeKey` 表示类型键，用于在 TVM 节点系统中标识该属性的类型。

- 在宏的定义中，首先使用 `static constexpr` 关键字将 `TypeKey` 赋值给了名为 `_type_key` 的静态常量字符指针。这样做的目的是确保 `_type_key` 的值在编译时就已经确定，并且只初始化一次。
- 接下来，通过调用 `TVM_DECLARE_FINAL_OBJECT_INFO` 宏来声明最终对象信息，其中包含了类名 `ClassName` 和基类 `::tvm::BaseAttrsNode`。这个宏的作用是将类名和基类传递给 TVM 的元编程系统，以便正确地生成派生类的元数据。
- 最后，通过模板函数 `_tvm_VisitAttrs` 来实现属性访问的机制。这个函数接受类型为 `FVisit` 的函数对象作为参数，用于遍历和访问属性。函数体中的注释 `// NOLINT(*)` 表示编译器在编译时应忽略该函数未被使用的错误警告。

第二个宏 `TVM_ATTR_FIELD` 用于声明属性字段。它也接受两个参数：`FieldName` 表示属性的字段名，`&FieldName` 表示对应的变量。

- 在宏的定义中，使用之前定义的模板函数 `_tvm_fvisit` 来处理属性字段的访问。`_tvm_fvisit(#FieldName, &FieldName)` 将属性字段的名称和对应的变量传递给 `_tvm_fvisit` 函数。这样，在后续的编译和执行过程中，就可以正确地访问和操作该属性字段了。

```{note}
在 `#FieldName` 中，`#` 符号是 C++ 中的预处理器指令的开始。它用于指示编译器将紧随其后的文本视为预处理指令。

在这种情况下，`#FieldName` 被视为宏定义的开始。宏定义是一种在编译之前进行文本替换的技术。通过使用 `#FieldName`，我们可以定义名为 `FieldName` 的宏。

在给定的代码片段中，`#FieldName` 被用作参数传递给宏 `TVM_ATTR_FIELD`。这意味着当该宏被调用时，`FieldName` 将被替换为实际的值。

总结起来，`#FieldName` 是 C++ 中预处理指令的语法，用于定义宏。在这个特定的代码片段中，它被用作参数传递给宏 `TVM_ATTR_FIELD`，并在宏展开时被替换为相应的值。
```

总结起来，这段代码定义了两个宏，用于在 TVM 中声明属性函数和属性字段。通过使用这些宏，可以在 C++ 代码中方便地声明和使用带有类型键的属性。

## `BaseAttrsNode`

`BaseAttrsNode` 类是所有属性类的基类。下面是对代码中各部分的解释：
- `virtual ~BaseAttrsNode() {}` 是虚析构函数，用于在删除派生类对象时进行适当的清理。
- `virtual void VisitAttrs(AttrVisitor* v) {}` 是纯虚函数，用于在派生类中实现访问属性的功能。
- `template <typename... Args> inline void InitBySeq(Args&&... args);` 是模板函数，用于通过一系列参数初始化属性。它使用了可变参数模板，可以接受任意数量的参数。
- `inline void PrintDocString(std::ostream& os) const;` 是内联函数，用于将文档字符串打印到给定的输出流中，并在末尾添加换行符。
- `virtual void VisitNonDefaultAttrs(AttrVisitor* v) = 0;` 是纯虚函数，用于在派生类中实现访问非默认属性的功能。
- `virtual Array<AttrFieldInfo> ListFieldInfo() const = 0;` 是纯虚函数，用于获取属性列表的信息。它返回包含属性字段信息的数组。
- `virtual void InitByPackedArgs(const TVMArgs& kwargs, bool allow_unknown = false) = 0;` 用于通过参数初始化属性的纯虚函数。该函数的作用是根据传入的键值对参数来初始化属性，并根据 `allow_unknown` 参数决定是否允许存在未知字段。如果所需字段不存在，则会抛出异常。